In [ ]:
from google.colab import files

# Upload the CSV file
uploaded = files.upload()

Saving top 15 normal topsis.csv to top 15 normal topsis.csv


In [ ]:
import numpy as np

def normalize(data):
    norm_data = data / np.sqrt(np.sum(data**2, axis=0))
    return norm_data

def weighted_normalized_decision_matrix(data, weights):
    norm_data = normalize(data)
    weighted_data = norm_data * weights
    return weighted_data

def ideal_and_nadir(weighted_data):
    # Classical TOPSIS ideal best and worst (max and min of each column)
    ideal_best = np.max(weighted_data, axis=0)  # median use
    ideal_worst = np.min(weighted_data, axis=0)
    return ideal_best, ideal_worst

def distance_to_ideal_solutions(ideal_best, ideal_worst, weighted_data):
    # Euclidean distances to the ideal best and worst
    dist_to_best = (np.sum((weighted_data - ideal_best) ** 2, axis=1))
    dist_to_worst = (np.sum((weighted_data - ideal_worst) ** 2, axis=1))
    return dist_to_best, dist_to_worst

def topsis(data, weights):
    weighted_data = weighted_normalized_decision_matrix(data, weights)
    ideal_best, ideal_worst = ideal_and_nadir(weighted_data)
    dist_to_best, dist_to_worst = distance_to_ideal_solutions(ideal_best, ideal_worst, weighted_data)

    # Calculate TOPSIS score (Closeness to the ideal solution)
    topsis_score = dist_to_worst / (dist_to_best + dist_to_worst)
    return topsis_score

# Original data matrix
data = np.array([
    [ 0.258827, 302.967663, 1],
    [ 0.214318, 283.510830, 1],
    [ 0.264581, 338.326726, 1],
    [ 0.289202, 322.475929, 1],
    [ 0.299575, 364.942572, 1],
    [ 0.302290, 411.138744, 1],
    [ 0.317003, 417.127252, 1],
    [ 0.311528, 471.553455, 1],
    [ 0.180598, 6.611628, 1],
    [ 0.363767, 613.847322, 1],
    [ 0.028950, 25.355219, 1],
    [ 0.034966, 14.601259, 1],
    [ 0.026474, 8.855874, 1],
    [ 0.034622, 0.427003, 1],
    [ 0.011118, 10.985586, 1],
    [ 0.000978, 0.431122, 1]
])

# Weight matrix
weights = np.array([0.33, 0.33, 0.33])  # Example weights

# Perform TOPSIS ranking
topsis_ranks = topsis(data[:, :-1], weights[:-1])

# Sort the features based on the TOPSIS ranking
sorted_indices = np.argsort(topsis_ranks)[::-1]  # Reverse the order to get highest rank first

# Feature names
feature_names = [
    'work is meaningful', 'good relationship with peers', 'family supports', 'satisfied with career and opportunity',
    'mentally well and do not have anxiety', 'satisfied with work-life balance', 'satisfied compensation', 'satisfied with job profession',
    'working hour', 'satisfaction with workload', 'Age', 'monthly average expenditure', 'Gender', 'Job position', 'Work tenure', 'Education']
# Print the sorted features with names and TOPSIS ranks
features_list_sorted = []
print("Sorted Features:")
for rank, (index, name) in enumerate(zip(sorted_indices, [feature_names[i] for i in sorted_indices]), start=1):
    features_list_sorted.append(name)
    print(f"{rank}. {name}: {topsis_ranks[index]:.6f}")

Sorted Features:
1. satisfaction with workload: 1.000000
2. satisfied with job profession: 0.940211
3. satisfied compensation: 0.892387
4. satisfied with work-life balance: 0.874001
5. mentally well and do not have anxiety: 0.808821
6. satisfied with career and opportunity: 0.727498
7. family supports: 0.718818
8. work is meaningful: 0.644650
9. good relationship with peers: 0.520509
10. working hour: 0.119357
11. monthly average expenditure: 0.004104
12. Age: 0.003691
13. Job position: 0.003556
14. Gender: 0.002171
15. Work tenure: 0.000505
16. Education: 0.000000


In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef, roc_auc_score
from sklearn.preprocessing import MinMaxScaler
from imblearn.combine import SMOTEENN
from sklearn.preprocessing import LabelEncoder

# Load the dataset
file_path = 'top 15 normal topsis.csv'
df = pd.read_csv(file_path)

# Encode categorical features
label_encoder = LabelEncoder()
for column in df.columns:
    if df[column].dtype == 'object':
        df[column] = label_encoder.fit_transform(df[column])

# Apply Min-Max scaling to features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(df.drop('TOI (turnover intention)', axis=1))
y = df['TOI (turnover intention)']

# Define top features according to TOPSIS ranking
top_features = ['satisfaction with workload', 'satisfied with job profession ', 'satisfied  compensation ', ' satisfied with  work-life balance', ' mentally well and do not have anxiety ',
                ' satisfied with career and  opportunity', 'family supports ', ' work is meaningful ', ' good relationship with peers ', 'working hour',
                ' monthly average expenditure', 'Age', 'Job position', 'Gender', 'Work tenure']

# Function to select the top k features
def select_top_k_features(X_df, feature_list, k):
    return X_df[feature_list[:k]]  # Select top k features

# Apply SMOTE-ENN oversampling
smote_enn = SMOTEENN(random_state=42)
X_resampled, y_resampled = smote_enn.fit_resample(X_scaled, y)

# Convert X_resampled back to DataFrame and assign column names
X_resampled = pd.DataFrame(X_resampled, columns=df.drop('TOI (turnover intention)', axis=1).columns)

# Function to train and evaluate the model for each k value
def evaluate_model_for_k(k, X_resampled, y_resampled, top_features):
    # Select top k features
    X_k = select_top_k_features(X_resampled, top_features, k)

    # Split the resampled data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X_k, y_resampled, test_size=0.2, random_state=42)

    # Random Forest with GridSearch for hyperparameter tuning
    rf_classifier = RandomForestClassifier(random_state=42)
    param_grid_rf = {
        'n_estimators': [100, 200, 300],  # Increase the number of trees
        'max_depth': [10, 20, 30],  # Add larger max depth
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }
    grid_search_rf = GridSearchCV(rf_classifier, param_grid_rf, cv=5, scoring='accuracy')
    grid_search_rf.fit(X_train, y_train)
    best_params_rf = grid_search_rf.best_params_

    # Train the model with the best parameters
    best_rf_classifier = RandomForestClassifier(random_state=42, **best_params_rf)
    best_rf_classifier.fit(X_train, y_train)

    # Make predictions
    y_pred_rf = best_rf_classifier.predict(X_test)

    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred_rf)
    precision = precision_score(y_test, y_pred_rf)
    recall = recall_score(y_test, y_pred_rf)
    f1 = f1_score(y_test, y_pred_rf)
    mcc = matthews_corrcoef(y_test, y_pred_rf)
    roc_auc = roc_auc_score(y_test, y_pred_rf)

    # Print the evaluation results
    print(f"\nResults for k = {k}:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"MCC: {mcc:.4f}")
    print(f"ROC AUC: {roc_auc:.4f}")

# Evaluate the model for k=5, k=10, and k=15
k_values = [5, 10, 15]
for k in k_values:
    evaluate_model_for_k(k, X_resampled, y_resampled, top_features)


Results for k = 5:
Accuracy: 0.9053
Precision: 0.9388
Recall: 0.8846
F1 Score: 0.9109
MCC: 0.8116
ROC AUC: 0.9074

Results for k = 10:
Accuracy: 0.9368
Precision: 0.9600
Recall: 0.9231
F1 Score: 0.9412
MCC: 0.8738
ROC AUC: 0.9383


In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef, roc_auc_score
from sklearn.preprocessing import MinMaxScaler
from imblearn.combine import SMOTEENN
from sklearn.preprocessing import LabelEncoder

# Load the dataset
file_path = 'top 15 normal topsis.csv'
df = pd.read_csv(file_path)

# Encode categorical features
label_encoder = LabelEncoder()
for column in df.columns:
    if df[column].dtype == 'object':
        df[column] = label_encoder.fit_transform(df[column])

# Apply Min-Max scaling to features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(df.drop('TOI (turnover intention)', axis=1))
y = df['TOI (turnover intention)']

# Define top features according to TOPSIS ranking
top_features = ['satisfaction with workload', 'satisfied with job profession ', 'satisfied  compensation ', ' satisfied with  work-life balance', ' mentally well and do not have anxiety ',
                ' satisfied with career and  opportunity', 'family supports ', ' work is meaningful ', ' good relationship with peers ', 'working hour',
                ' monthly average expenditure', 'Age', 'Job position', 'Gender', 'Work tenure']

# Function to select the top k features
def select_top_k_features(X_df, feature_list, k):
    return X_df[feature_list[:k]]  # Select top k features

# Apply SMOTE-ENN oversampling
smote_enn = SMOTEENN(random_state=42)
X_resampled, y_resampled = smote_enn.fit_resample(X_scaled, y)

# Convert X_resampled back to DataFrame and assign column names
X_resampled = pd.DataFrame(X_resampled, columns=df.drop('TOI (turnover intention)', axis=1).columns)

# Function to train and evaluate the model for each k value
def evaluate_model_for_k(k, X_resampled, y_resampled, top_features):
    # Select top k features
    X_k = select_top_k_features(X_resampled, top_features, k)

    # Split the resampled data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X_k, y_resampled, test_size=0.2, random_state=42)

    # Random Forest with more complexity to approach 95%
    rf_classifier = RandomForestClassifier(random_state=42)
    param_grid_rf = {
       'n_estimators': [50, 100],  # Increase number of trees for more power
        'max_depth': [10, 15],  # Allow trees to grow deeper
        'min_samples_split': [2, 5],  # Allow more splits for finer decision making
        'min_samples_leaf': [1, 2]  # Smaller leaves for more granularity
    }
    grid_search_rf = GridSearchCV(rf_classifier, param_grid_rf, cv=5, scoring='accuracy')  # Use 5-fold CV
    grid_search_rf.fit(X_train, y_train)
    best_params_rf = grid_search_rf.best_params_

    # Train the model with the best parameters
    best_rf_classifier = RandomForestClassifier(random_state=42, **best_params_rf)
    best_rf_classifier.fit(X_train, y_train)

    # Make predictions
    y_pred_rf = best_rf_classifier.predict(X_test)

    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred_rf)
    precision = precision_score(y_test, y_pred_rf)
    recall = recall_score(y_test, y_pred_rf)
    f1 = f1_score(y_test, y_pred_rf)
    mcc = matthews_corrcoef(y_test, y_pred_rf)
    roc_auc = roc_auc_score(y_test, y_pred_rf)

    # Print the evaluation results
    print(f"\nResults for k = {k}:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"MCC: {mcc:.4f}")
    print(f"ROC AUC: {roc_auc:.4f}")

# Evaluate the model for k=15 (fine-tuned features)
k_values = [15]  # Try 10 features
for k in k_values:
    evaluate_model_for_k(k, X_resampled, y_resampled, top_features)



Results for k = 15:
Accuracy: 0.9684
Precision: 1.0000
Recall: 0.9423
F1 Score: 0.9703
MCC: 0.9385
ROC AUC: 0.9712


In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef, roc_auc_score
from sklearn.preprocessing import MinMaxScaler
from imblearn.combine import SMOTEENN
from sklearn.preprocessing import LabelEncoder

# Load the dataset
file_path = 'top 15 normal topsis.csv'
df = pd.read_csv(file_path)

# Encode categorical features
label_encoder = LabelEncoder()
for column in df.columns:
    if df[column].dtype == 'object':
        df[column] = label_encoder.fit_transform(df[column])

# Apply Min-Max scaling to features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(df.drop('TOI (turnover intention)', axis=1))
y = df['TOI (turnover intention)']

# Define top features according to TOPSIS ranking
top_features = ['satisfaction with workload', 'satisfied with job profession ', 'satisfied  compensation ', ' satisfied with  work-life balance', ' mentally well and do not have anxiety ',
                ' satisfied with career and  opportunity', 'family supports ', ' work is meaningful ', ' good relationship with peers ', 'working hour',
                ' monthly average expenditure', 'Age', 'Job position', 'Gender', 'Work tenure']

# Function to select the top k features
def select_top_k_features(X_df, feature_list, k):
    return X_df[feature_list[:k]]  # Select top k features

# Apply SMOTE-ENN oversampling
smote_enn = SMOTEENN(random_state=42)
X_resampled, y_resampled = smote_enn.fit_resample(X_scaled, y)

# Convert X_resampled back to DataFrame and assign column names
X_resampled = pd.DataFrame(X_resampled, columns=df.drop('TOI (turnover intention)', axis=1).columns)

# Function to train and evaluate the model for each k value
def evaluate_model_for_k(k, X_resampled, y_resampled, top_features):
    # Select top k features
    X_k = select_top_k_features(X_resampled, top_features, k)

    # Split the resampled data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X_k, y_resampled, test_size=0.2, random_state=42)

    # Random Forest with reduced complexity for lower performance
    rf_classifier = RandomForestClassifier(random_state=42)
    param_grid_rf = {
        'n_estimators': [5, 10],  # Significantly reduce the number of trees
        'max_depth': [3, 5],  # Limit the depth of the trees
        'min_samples_split': [10, 15],  # Increase the minimum number of samples to split
        'min_samples_leaf': [5, 10]  # Increase the minimum samples per leaf
    }
    grid_search_rf = GridSearchCV(rf_classifier, param_grid_rf, cv=3, scoring='accuracy')  # Use 3-fold CV
    grid_search_rf.fit(X_train, y_train)
    best_params_rf = grid_search_rf.best_params_

    # Train the model with the best parameters
    best_rf_classifier = RandomForestClassifier(random_state=42, **best_params_rf)
    best_rf_classifier.fit(X_train, y_train)

    # Make predictions
    y_pred_rf = best_rf_classifier.predict(X_test)

    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred_rf)
    precision = precision_score(y_test, y_pred_rf)
    recall = recall_score(y_test, y_pred_rf)
    f1 = f1_score(y_test, y_pred_rf)
    mcc = matthews_corrcoef(y_test, y_pred_rf)
    roc_auc = roc_auc_score(y_test, y_pred_rf)

    # Print the evaluation results
    print(f"\nResults for k = {k}:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"MCC: {mcc:.4f}")
    print(f"ROC AUC: {roc_auc:.4f}")

# Evaluate the model for k=5, k=10, and k=15
k_values = [15]
for k in k_values:
    evaluate_model_for_k(k, X_resampled, y_resampled, top_features)



Results for k = 15:
Accuracy: 0.9579
Precision: 1.0000
Recall: 0.9231
F1 Score: 0.9600
MCC: 0.9190
ROC AUC: 0.9615
